In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PATH = "Resultados_ICFES_Cordoba_clean.csv"  # <-- CAMBIA ESTO
df = pd.read_csv(PATH, low_memory=False)
print("Shape:", df.shape)
df.head()

Shape: (194259, 51)


,periodo,estu_tipodocumento,estu_consecutivo,cole_area_ubicacion,cole_bilingue,cole_calendario,cole_caracter,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_cod_depto_ubicacion,...,fami_tienecomputador,fami_tieneinternet,fami_tienelavadora,desemp_ingles,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global
0,20224,TI,SB11202240331749,URBANO,N,A,ACADÉMICO,1.230900e+11,123090000301,23,...,NO,NO,SI,A-,43.0,50.0,42.0,46.0,52.0,236.0
1,20224,TI,SB11202240555914,RURAL,N,A,ACADÉMICO,2.231620e+11,223162001662,23,...,NO,NO,SI,A2,59.0,54.0,51.0,57.0,49.0,266.0
2,20224,TI,SB11202240559153,RURAL,NaN,A,NO APLICA,2.236700e+11,223670000507,23,...,NO,NO,NO,A-,34.0,37.0,28.0,36.0,41.0,177.0
3,20224,TI,SB11202240526915,RURAL,NaN,A,ACADÉMICO,2.236600e+11,223660001084,23,...,NaN,NaN,NaN,A-,41.0,63.0,63.0,58.0,60.0,297.0
4,20224,TI,SB11202240311533,URBANO,N,A,ACADÉMICO,3.230010e+11,323001004178,23,...,SI,SI,SI,A1,56.0,72.0,52.0,59.0,65.0,308.0


In [3]:
# si no hay puntaje global, no se puede usar el registro
df = df.dropna(subset=["punt_global"])

# si no tiene estrato, no se puede usar el registro
df = df.dropna(subset=["fami_estratovivienda"])

# eliminar si el regiutro de puntaje es "SIN ESTRATO" como string
df = df[df["fami_estratovivienda"] != "SIN ESTRATO"]


In [4]:
df.info()

# Porcentaje de missing por columna (top 30)
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
print(missing_pct.head(30))


<class 'pandas.DataFrame'>
Index: 111184 entries, 0 to 194255
Data columns (total 51 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   periodo                        111184 non-null  int64  
 1   estu_tipodocumento             111184 non-null  str    
 2   estu_consecutivo               111184 non-null  str    
 3   cole_area_ubicacion            111184 non-null  str    
 4   cole_bilingue                  89273 non-null   str    
 5   cole_calendario                111184 non-null  str    
 6   cole_caracter                  110634 non-null  str    
 7   cole_cod_dane_establecimiento  111184 non-null  float64
 8   cole_cod_dane_sede             111184 non-null  int64  
 9   cole_cod_depto_ubicacion       111184 non-null  int64  
 10  cole_cod_mcpio_ubicacion       111184 non-null  int64  
 11  cole_codigo_icfes              111184 non-null  float64
 12  cole_depto_ubicacion           111184 non-null

In [5]:
df_anova = df[df["fami_estratovivienda"] != "SIN ESTRATO"]

from scipy import stats

grupos = [
    group["punt_global"].dropna()
    for name, group in df_anova.groupby("fami_estratovivienda")
]

f_stat, p_value = stats.f_oneway(*grupos)

print("F-statistic:", f_stat)
print("p-value:", p_value)


F-statistic: 399.62941324452686
p-value: 0.0


In [6]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(
    endog=df_anova["punt_global"],
    groups=df_anova["fami_estratovivienda"],
    alpha=0.05
)

print(tukey)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1    group2  meandiff p-adj   lower    upper  reject
-----------------------------------------------------------
ESTRATO 1 ESTRATO 2  14.5257    0.0  13.4832 15.5682   True
ESTRATO 1 ESTRATO 3  13.1716    0.0  11.5044 14.8388   True
ESTRATO 1 ESTRATO 4   6.5297    0.0    3.713  9.3465   True
ESTRATO 1 ESTRATO 5  11.6362    0.0   7.4885  15.784   True
ESTRATO 1 ESTRATO 6  17.4133    0.0  12.1721 22.6544   True
ESTRATO 2 ESTRATO 3  -1.3542    0.3  -3.2132  0.5048  False
ESTRATO 2 ESTRATO 4   -7.996    0.0 -10.9304 -5.0617   True
ESTRATO 2 ESTRATO 5  -2.8895 0.3732   -7.118   1.339  False
ESTRATO 2 ESTRATO 6   2.8875  0.631  -2.4177  8.1928  False
ESTRATO 3 ESTRATO 4  -6.6419    0.0  -9.8517  -3.432   True
ESTRATO 3 ESTRATO 5  -1.5354 0.9216  -5.9595  2.8888  False
ESTRATO 3 ESTRATO 6   4.2417 0.2317  -1.2208  9.7041  False
ESTRATO 4 ESTRATO 5   5.1065 0.0401   0.1338 10.0792   True
ESTRATO 4 ESTRATO 6  10.8835    0.0   4.